In [5]:
"""deluca.agents._mem"""
from numbers import Real
from typing import Callable

import jax
import jax.numpy as jnp
import numpy as np
from jax import grad
from jax import jit

from deluca.agents._lqr import LQR
from deluca.agents.core import Agent

from deluca.utils.optimR import optimROBD

In [8]:
def idx(B):
    return np.where(B.any(axis=1))[0] #return indices of non-zero rows

In [21]:
'''Test idx'''
B = np.array([[0, 0, 0], [0,0,0], [1,2,3], [0, 1, 0], [0,0,0], [0,0,0],[1,0,0]])
ks = idx(B)
print(ks)
print("works!")

[2 3 6]
works!


In [27]:
def defineP(ks):
    ps = np.ndarray((len(ks), 1))
    ps[0] = ks[0]
    for i in range(1,len(ks)):
        ps[i] = ks[i] - ks[i-1]
    return ps.astype(int), int(np.amax(ps))

In [28]:
''' Test defineP '''
ps, p = defineP(idx(B))
print(ps, p)

[[2]
 [1]
 [3]] 3


In [57]:
#TODO: define d and other undefined variables throughout
#TODO: check indices all over (1-indexed vs 0-indexed)
def defineCs(A, ks, p, ps):
    d = len(ks)
    Cs = np.ndarray((p, d, d))
    #TODO: change from 0-indexed to 1-indexed (add dummy)
    for i in range(0, p):
        a_identity = A[ks, :]
        C = np.ndarray((d, d))
        for j in range(0, d):
            if i <= ps[j]:
                print(ks[j])
                C[:, j] = a_identity[:, ks[j]-i-1]
            else:
                C[:, j] = np.zeros(d)
        Cs[i] = C
    return Cs

In [58]:
''' Testing defineCs '''
defineCs(B, ks, p, ps)

2
3
6


IndexError: index 5 is out of bounds for axis 1 with size 3

In [59]:
def hitFunc(qs, ps, ks, t):
    # define a function of y that can be optimized
    # is y n-dimensional or d-dimensional? 
    # TODO: clarify and change to d-dimensional based on ks
    def func(y):
        masterSum = 0
        d = len(ks)
        for i in range(d):
            littleSum = 0
            for j in range(1, ps[i]):
                littleSum += qs[t+j]
            masterSum += (littleSum * (y[ks[i]] ** 2))
        return masterSum/2

    return func